In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Deep learning school/HW6/packages')

import dataset
import models
import torch
import torchvision
from torchvision import transforms
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

Mounted at /content/drive


In [3]:
!unzip -q "/content/drive/MyDrive/Deep learning school/HW6/horse2zebra.zip"

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset_horses = dataset.FolderDataSet("./horse2zebra/test/testA", transform=transform)
dataloader_horses = torch.utils.data.DataLoader(dataset_horses, batch_size = 1, shuffle=False)
dataset_zebras = dataset.FolderDataSet("./horse2zebra/test/testB", transform=transform)
dataloader_zebras = torch.utils.data.DataLoader(dataset_zebras, batch_size = 1, shuffle=False)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

pool_images = None
pool_targets = None
gan = models.CycleGAN(lambd = 10, resnet_layers = 6, device = device, buffer_images=pool_images, dim_gen = 32, buffer_targets=pool_targets)
gan.to(device)
checkpoint = torch.load('/content/drive/MyDrive/Deep learning school/HW6/horse2zebra_weights')
gan.load_state_dict(checkpoint['gan_state_dict'])

/content/drive/MyDrive/Deep learning school/HW6/packages/models.py:9: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(x.weight, m, s)


<All keys matched successfully>

In [5]:
from PIL import Image  
import PIL
gan.eval()
for i, img_horse in enumerate(dataloader_horses):
    img_horse = img_horse.to(device)
    converted_horse = torchvision.transforms.ToPILImage()(dataset.de_norm(gan(img_horse)[0].cpu()))
    converted_horse.save('/content/drive/MyDrive/Deep learning school/HW6/horse2zebra_results/horse2zebra/' + str(i) + '.png')

for i, img_zebra in enumerate(dataloader_zebras):
    img_zebra = img_zebra.to(device)
    converted_zebra = torchvision.transforms.ToPILImage()(dataset.de_norm(gan(img_zebra, inverse = True)[0].cpu()))
    converted_zebra.save('/content/drive/MyDrive/Deep learning school/HW6/horse2zebra_results/zebra2horse/' + str(i) + '.png')
